# Kickstarter Projects
### Rebecca Tung (1448196)
### Sidney Fox (1524992)

In [74]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [75]:
#Libraries used throughout the Project
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import math
import requests
import io
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [76]:
#Reference to .csv file
#url = 'https://raw.githubusercontent.com/stfox13/CSC478FinalProject/master/Data/ks-projects-201612.csv'

#Read 2016 projects in as pandas DataFrame
#df = pd.read_csv(url, index_col = 0)
df16 = pd.read_csv("ks-projects-201612.csv")
df18 = pd.read_csv("ks-projects-201801.csv")
#Remove the last three columns of the 
#df = df[list(df)[:-4]]

#Remove trailing white space from header
df16.columns = df16.columns.str.strip().str.replace(' ','_').str.replace(':','')
df18.columns = df18.columns.str.strip().str.replace(' ','_').str.replace(':','')

In [77]:
df16 = df16[list(df16)[:-2]]
df16.columns
df18.columns

Index([u'ID', u'name', u'category', u'main_category', u'currency', u'deadline',
       u'goal', u'launched', u'pledged', u'state', u'backers', u'country',
       u'usd_pledged', u'Unnamed_13', u'Unnamed_14'],
      dtype='object')

Index([u'ID', u'name', u'category', u'main_category', u'currency', u'deadline',
       u'goal', u'launched', u'pledged', u'state', u'backers', u'country',
       u'usd_pledged', u'usd_pledged_real', u'usd_goal_real'],
      dtype='object')

In [78]:
df16.shape
df16.head(3)
df18.shape
df18.head(3)

(323750, 15)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,Unnamed_13,Unnamed_14
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,NaN,NaN


(378661, 15)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00


In [79]:
df16.sort_values(['ID']).head(5)
df18.sort_values(['ID']).head(5)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,Unnamed_13,Unnamed_14
256658,5971,An American Apocalypse: The Reckoning,Comics,Comics,USD,2012-08-06 20:00:00,48000,2012-06-08 18:46:56,376,failed,7,US,376,NaN,NaN
143377,18520,Grandma's are Life,World Music,Music,USD,2016-11-18 15:32:40,15000,2016-10-19 15:32:40,62,failed,4,US,0,NaN,NaN
186586,21109,Meta,Performance Art,Art,GBP,2015-05-06 23:00:00,150,2015-04-08 00:37:44,173,successful,11,GB,258.03603248,NaN,NaN
190996,21371,Water Powered Car,Technology,Technology,USD,2016-08-10 19:02:01,75000,2016-07-26 19:02:01,120,failed,4,US,50,NaN,NaN
197630,24380,Puss N' Books: A relaxing cat cafe and bookstore.,Spaces,Food,USD,2015-11-26 17:25:33,20000,2015-10-27 16:25:33,776,failed,18,US,776,NaN,NaN


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real,usd_goal_real
299904,5971,An American Apocalypse: The Reckoning,Comics,Comics,USD,2012-08-06,48000.0,2012-06-08 18:46:56,376.0,failed,7,US,376.00,376.00,48000.0
167485,18520,Grandma's are Life,World Music,Music,USD,2016-11-18,15000.0,2016-10-19 15:32:40,62.0,failed,4,US,0.00,62.00,15000.0
218065,21109,Meta,Performance Art,Art,GBP,2015-05-06,150.0,2015-04-08 00:37:44,173.0,successful,11,GB,258.04,266.65,231.2
223206,21371,Water Powered Car,Technology,Technology,USD,2016-08-10,75000.0,2016-07-26 19:02:01,120.0,failed,4,US,50.00,120.00,75000.0
230898,24380,Puss N' Books: A relaxing cat cafe and bookstore.,Spaces,Food,USD,2015-11-26,20000.0,2015-10-27 16:25:33,776.0,failed,18,US,776.00,776.00,20000.0


In [80]:
df16.isnull().sum()
df18.isnull().sum()

ID                    0
name                  2
category              5
main_category         0
currency              0
deadline              0
goal                  0
launched              0
pledged               0
state                 0
backers               0
country               0
usd_pledged        3790
Unnamed_13       323125
Unnamed_14       323738
dtype: int64

ID                     0
name                   4
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd_pledged         3797
usd_pledged_real       0
usd_goal_real          0
dtype: int64

In [81]:
df18.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                float64
launched             object
pledged             float64
state                object
backers               int64
country              object
usd_pledged         float64
usd_pledged_real    float64
usd_goal_real       float64
dtype: object

In [82]:
df16[(df16['name'].isnull())]
df16[(df16['ID'] == 1848699072) | (df16['ID'] == 634871725)]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,Unnamed_13,Unnamed_14
265251,648853978,NaN,Product Design,Design,USD,2016-07-18 05:01:47,2500,2016-06-18 05:01:47,0,suspended,0,US,0,NaN,NaN
289847,796533179,NaN,Painting,Art,USD,2011-12-05 05:59:00,35000,2011-11-06 23:55:55,220,failed,5,US,220,NaN,NaN


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,Unnamed_13,Unnamed_14
142830,1848699072,n/a,Narrative Film,Film & Video,USD,2012-02-29 15:04:00,200000,2012-01-01 12:35:31,100,failed,1,US,100,NaN,NaN
262901,634871725,null,Video Games,Games,GBP,2013-01-06 23:00:00,2000,2012-12-19 23:57:48,196,failed,12,GB,317.7284362,NaN,NaN


In [83]:
df18[(df18['name'].isnull())]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged,usd_pledged_real,usd_goal_real
166851,1848699072,NaN,Narrative Film,Film & Video,USD,2012-02-29,200000.0,2012-01-01 12:35:31,100.0,failed,1,US,100.00,100.00,200000.00
307234,634871725,NaN,Video Games,Games,GBP,2013-01-06,2000.0,2012-12-19 23:57:48,196.0,failed,12,GB,317.73,316.05,3224.97
309991,648853978,NaN,Product Design,Design,USD,2016-07-18,2500.0,2016-06-18 05:01:47,0.0,suspended,0,US,0.00,0.00,2500.00
338931,796533179,NaN,Painting,Art,USD,2011-12-05,35000.0,2011-11-06 23:55:55,220.0,failed,5,US,220.00,220.00,35000.00


In [86]:
# Check the difference between these two datasets
diff = set(df18.ID) - set(df16.ID)
len(diff)
diff2 = set(df16.ID) - set(df18.ID)
len(diff2)
#len(df18) - len(df16)

54911

0